<a href="https://colab.research.google.com/github/ettoreotery/AMD-project-DSE-25/blob/main/AMD_Ettore_Oteri_v3_1_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
os.environ['KAGGLE_USERNAME'] = "ettoreoteri"
os.environ['KAGGLE_KEY'] = "xxxxxxxxxxxxxxxxxxxxxxxxxxx"
!kaggle datasets download -d mohamedbakhet/amazon-books-reviews

!unzip -q "*.zip" -d /content/

Dataset URL: https://www.kaggle.com/datasets/mohamedbakhet/amazon-books-reviews
License(s): CC0-1.0
 96% 1.02G/1.06G [00:00<00:00, 1.23GB/s]
100% 1.06G/1.06G [00:00<00:00, 1.21GB/s]


In [ ]:
import pandas as pd
df1 = pd.read_csv("Books_rating.csv")
df1.head()

,Id,Title,Price,User_id,profileName,review/helpfulness,review/score,review/time,review/summary,review/text
0,1882931173,Its Only Art If Its Well Hung!,NaN,AVCGYZL8FQQTD,"Jim of Oz ""jim-of-oz""",7/7,4.0,940636800,Nice collection of Julie Strain images,This is only for Julie Strain fans. It's a col...
1,0826414346,Dr. Seuss: American Icon,NaN,A30TK6U7DNS82R,Kevin Killian,10/10,5.0,1095724800,Really Enjoyed It,I don't care much for Dr. Seuss but after read...
2,0826414346,Dr. Seuss: American Icon,NaN,A3UH4UZ4RSVO82,John Granger,10/11,5.0,1078790400,Essential for every personal and Public Library,"If people become the books they read and if ""t..."
3,0826414346,Dr. Seuss: American Icon,NaN,A2MVUWT453QH61,"Roy E. Perry ""amateur philosopher""",7/7,4.0,1090713600,Phlip Nel gives silly Seuss a serious treatment,"Theodore Seuss Geisel (1904-1991), aka &quot;D..."
4,0826414346,Dr. Seuss: American Icon,NaN,A22X4XUPKF66MR,"D. H. Richards ""ninthwavestore""",3/3,4.0,1107993600,Good academic overview,Philip Nel - Dr. Seuss: American IconThis is b...


In [ ]:
print(f"This dataset contains {df1.shape[0]} rows and {df1.shape[1]} columns.")

This dataset contains 3000000 rows and 10 columns.


In [ ]:
!pip install datasketch

import string
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import nltk
from itertools import combinations
from datasketch import MinHash, MinHashLSH

# Download the required NLTK resources
nltk.download('punkt', quiet=True)
nltk.download('stopwords', quiet=True)
nltk.download('wordnet', quiet=True)
nltk.download('omw-1.4', quiet=True)
nltk.download('punkt_tab', quiet=True)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.2/89.2 kB 2.5 MB/s eta 0:00:00


True

In [ ]:
# Tokenization function
def safe_tokenize(text):
    try:
        return word_tokenize(text)
    except:
        return text.split()  # Simple fallback

# Text preprocessing setup
lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words('english'))
translator = str.maketrans('', '', string.punctuation)

def preprocess_text(text):
    if not isinstance(text, str):
        return []

    # Text normalization
    text = text.lower().translate(translator)
    tokens = safe_tokenize(text)
    # Return sorted tuple of unique lemmatized tokens
    return tuple(sorted(set(lemmatizer.lemmatize(word) for word in tokens
                      if word not in stop_words and len(word) > 2)))

# Data loading and cleaning
print("Loading and cleaning data...")
df = pd.read_csv("Books_rating.csv")

# Remove exact duplicates
df = df.drop_duplicates(subset=['review/text'], keep='first')

# Sample 10,000 reviews and preprocess
df = df.sample(10000, random_state=36)
df['tokens'] = df['review/text'].apply(preprocess_text)

# Remove duplicates after preprocessing
df = df.drop_duplicates(subset=['tokens'], keep='first')

# Filter reviews with at least 5 unique tokens
df = df[df['tokens'].apply(len) >= 5]
reviews = df['tokens'].tolist()

Loading and cleaning data...


In [ ]:
# Similarity calculation using MinHash + LSH
print("Calculating similarities with MinHash...")

minhashes = []
num_perm = 128
for tokens in reviews:
    m = MinHash(num_perm=num_perm)
    for token in tokens:
        m.update(token.encode('utf8'))
    minhashes.append(m)

lsh = MinHashLSH(threshold=0.5, num_perm=num_perm)
for i, m in enumerate(minhashes):
    lsh.insert(str(i), m)

results = []
for i, mh1 in enumerate(minhashes):
    for j_str in lsh.query(mh1):
        j = int(j_str)
        if j > i:
            set1 = set(reviews[i])
            set2 = set(reviews[j])
            intersection = set1 & set2
            union = set1 | set2
            jaccard = len(intersection) / len(union) if union else 0.0
            if jaccard > 0:
                results.append((jaccard, i, j))

# Sort and select top 20 pairs
results.sort(reverse=True, key=lambda x: x[0])
top_20 = results[:20]

# Display results
print("\nTOP 20 MOST SIMILAR PAIRS:")
for rank, (sim, i, j) in enumerate(top_20, 1):
    orig_text_i = df.iloc[i]['review/text'][:100] + ('...' if len(df.iloc[i]['review/text']) > 100 else '')
    orig_text_j = df.iloc[j]['review/text'][:100] + ('...' if len(df.iloc[j]['review/text']) > 100 else '')

    print(f"\n#{rank}: Similarity = {sim:.4f}")
    print(f"Review {i}: {orig_text_i}")
    print(f"Review {j}: {orig_text_j}")
    print(f"Common tokens: {set(reviews[i]) & set(reviews[j])}")

# Save results
results_df = pd.DataFrame(top_20, columns=['similarity', 'index1', 'index2'])
results_df['text1'] = results_df['index1'].apply(lambda x: df.iloc[x]['review/text'])
results_df['text2'] = results_df['index2'].apply(lambda x: df.iloc[x]['review/text'])
results_df.to_csv('top_pairs_no_duplicates.csv', index=False)

Calculating similarities with MinHash...

TOP 20 MOST SIMILAR PAIRS:

#1: Similarity = 0.5000
Review 6795: One of the best books of photojournalism that I've ever seen. Highly recommended.
Review 9946: One of the best books I've ever read!.
Common tokens: {'one', 'ever', 'best', 'ive', 'book'}

#2: Similarity = 0.4444
Review 5170: I received this book and it was in perfect condition, it came very quickly.
Review 5546: The book was in perfect condition when I received it. It is an interesting read, but it was too long...
Common tokens: {'received', 'book', 'perfect', 'condition'}

#3: Similarity = 0.4000
Review 6224: this is the best book ever. it is very funny. I have and have read all the books. they are very good...
Review 9946: One of the best books I've ever read!.
Common tokens: {'read', 'ever', 'best', 'book'}

#4: Similarity = 0.3636
Review 8672: Wonderful service. Arrived in 6 days. Book in condition as promised.
Review 9713: The book arrived in a great condition. Also, it arri